# Electrical Panel Pattern Recognition - Exploratory POC

This notebook explores using Weaviate vector database with CLIP embeddings to identify patterns in electrical panel images.

**Goals:**
- Load electrical panel images into Weaviate
- Use vector similarity search to find compatible panels
- Identify incompatible panels based on visual patterns
- Build foundation for capacity analysis

## Setup

**Prerequisites:**
1. Start Weaviate: `docker compose up`
2. Install dependencies below

In [1]:
%pip install weaviate-client

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle, json, os, IPython, base64
import weaviate
import weaviate.classes as wvc
from weaviate.classes.config import Configure, Property, DataType
from PIL import Image
import io

## Connect to Weaviate

Connect to the locally running Weaviate instance (meter-image-weaviate container)

In [3]:
client = weaviate.connect_to_local()

print(f"Client created? {client.is_ready()}")
if client.is_ready():
    print("✓ Successfully connected to Weaviate")
else:
    print("✗ Connection failed - make sure Weaviate is running (docker compose up)")

Client created? True
✓ Successfully connected to Weaviate


## Create Electrical Panel Collection

Define a schema for storing electrical panel images with metadata

In [ ]:
# Clean up any existing collection
if client.collections.exists("ElectricalPanel"):
    client.collections.delete("ElectricalPanel")
    print("Deleted existing ElectricalPanel collection")

In [ ]:
# Create collection with multi2vec-clip for image embeddings
client.collections.create(
    name="ElectricalPanel",
    vectorizer_config=Configure.Vectorizer.multi2vec_clip(
        image_fields=["image"]
    ),
    properties=[
        Property(name="image", data_type=DataType.BLOB),
        Property(name="filename", data_type=DataType.TEXT),
        Property(name="panel_type", data_type=DataType.TEXT),  # e.g., "main", "sub", "meter"
        Property(name="compatibility_tag", data_type=DataType.TEXT),  # e.g., "residential", "commercial"
        Property(name="notes", data_type=DataType.TEXT)  # Additional observations
    ]
)

print("✓ ElectricalPanel collection created")

## Load Panel Images

Load electrical panel images from the Images/ directory into Weaviate

In [ ]:
collection = client.collections.get("ElectricalPanel")

# Supported image formats
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

# Get all image files
image_dir = "Images/"
image_files = [f for f in os.listdir(image_dir) if os.path.splitext(f)[1].lower() in image_extensions]

print(f"Found {len(image_files)} images in {image_dir}")
print("\nLoading images into Weaviate...")

success_count = 0
error_count = 0

for img in image_files:
    try:
        with open(f"{image_dir}{img}", "rb") as file:
            image_data = base64.b64encode(file.read()).decode('utf-8')
        
        # Insert with basic metadata (can be enhanced later)
        collection.data.insert(
            properties={
                "image": image_data,
                "filename": img,
                "panel_type": "unknown",  # To be classified
                "compatibility_tag": "untagged",  # To be labeled
                "notes": "Initial import"
            }
        )
        success_count += 1
        if success_count % 10 == 0:
            print(f"  Loaded {success_count} images...")
    except Exception as e:
        print(f"✗ Error adding {img}: {e}")
        error_count += 1

print(f"\n✓ Successfully loaded: {success_count} images")
if error_count > 0:
    print(f"✗ Failed to load: {error_count} images")

## Query 1: Text-based Pattern Search

Search for panels based on textual descriptions

In [ ]:
def search_panels_by_text(query_text, limit=5):
    """Search for electrical panels using text description"""
    collection = client.collections.get("ElectricalPanel")
    
    res = collection.query.near_text(
        query=query_text,
        limit=limit
    )
    
    results = []
    for obj in res.objects:
        results.append({
            "filename": obj.properties.get("filename"),
            "panel_type": obj.properties.get("panel_type"),
            "compatibility_tag": obj.properties.get("compatibility_tag")
        })
    
    return results

# Example searches
print("Search: 'electrical meter panel'")
results = search_panels_by_text("electrical meter panel", limit=3)
print(json.dumps(results, indent=2))

In [ ]:
# Visualize top result
if results and results[0]['filename']:
    display(Image.open(f"Images/{results[0]['filename']}"))
    print(f"Top match: {results[0]['filename']}")

## Query 2: Image-based Similarity Search

Find similar panels based on a reference image

In [ ]:
def find_similar_panels(query_image_path, limit=5):
    """Find panels visually similar to the query image"""
    collection = client.collections.get("ElectricalPanel")
    
    # Read and encode query image
    with open(query_image_path, "rb") as file:
        query_image = base64.b64encode(file.read()).decode('utf-8')
    
    # Search for similar images
    res = collection.query.near_image(
        near_image=query_image,
        limit=limit
    )
    
    results = []
    for obj in res.objects:
        results.append({
            "filename": obj.properties.get("filename"),
            "panel_type": obj.properties.get("panel_type"),
            "compatibility_tag": obj.properties.get("compatibility_tag")
        })
    
    return results

In [ ]:
# Example: Find panels similar to a reference image
# Update this path to point to one of your panel images
reference_image = "Images/" + image_files[0]  # Using first image as example

print(f"Finding panels similar to: {os.path.basename(reference_image)}")
print("\nReference image:")
display(Image.open(reference_image))

similar = find_similar_panels(reference_image, limit=4)
print("\nTop similar panels:")
print(json.dumps(similar, indent=2))

In [ ]:
# Visualize similar panels
print("Similar panels (excluding self):")
for i, result in enumerate(similar[1:4], 1):  # Skip first result (self-match)
    if result['filename']:
        print(f"\n{i}. {result['filename']}")
        display(Image.open(f"Images/{result['filename']}"))

## Query 3: Identify Compatible vs Incompatible Panels

Search for panels that match or differ from a reference configuration

In [ ]:
def find_compatible_panels(reference_image_path, compatibility_threshold=0.7):
    """
    Find panels compatible with reference (high similarity)
    Note: Distance/similarity scores would need Weaviate metadata to filter precisely
    This is a simplified version using top results
    """
    print("🔍 COMPATIBLE PANELS (High Similarity)")
    print("="*60)
    
    similar = find_similar_panels(reference_image_path, limit=6)
    
    # Top results are most compatible (excluding self-match)
    compatible = similar[1:4]  # Get top 3 matches after self
    
    for i, panel in enumerate(compatible, 1):
        print(f"{i}. {panel['filename']}")
    
    return compatible

def find_incompatible_panels(reference_image_path):
    """
    Find panels incompatible with reference (low similarity)
    Using text query to find dissimilar panel types
    """
    print("\n⚠️  INCOMPATIBLE PANELS (Low Similarity)")
    print("="*60)
    
    # Strategy: Search for panels with different characteristics
    # This is exploratory - would need labeled data for real classification
    all_panels = find_similar_panels(reference_image_path, limit=20)
    
    # Bottom results are least similar
    incompatible = all_panels[-3:]  # Get last 3 (least similar)
    
    for i, panel in enumerate(incompatible, 1):
        print(f"{i}. {panel['filename']}")
    
    return incompatible

In [ ]:
# Run compatibility analysis
reference_panel = "Images/" + image_files[0]

print(f"REFERENCE PANEL: {os.path.basename(reference_panel)}")
print("="*60)
display(Image.open(reference_panel))

compatible = find_compatible_panels(reference_panel)
incompatible = find_incompatible_panels(reference_panel)

In [ ]:
# Visualize compatible panels
print("\n🔍 COMPATIBLE PANELS:")
print("="*60)
for panel in compatible:
    if panel['filename']:
        print(f"\n{panel['filename']}")
        display(Image.open(f"Images/{panel['filename']}"))

In [ ]:
# Visualize incompatible panels
print("\n⚠️  INCOMPATIBLE PANELS:")
print("="*60)
for panel in incompatible:
    if panel['filename']:
        print(f"\n{panel['filename']}")
        display(Image.open(f"Images/{panel['filename']}"))

## Interactive Exploration

Use this cell to run custom queries and explore the dataset

In [ ]:
# Custom query examples - modify as needed

# Text search
query = "circuit breaker panel"  # Try: "meter", "breaker", "electrical box"
results = search_panels_by_text(query, limit=3)
print(f"Results for '{query}':")
for r in results:
    print(f"  - {r['filename']}")
    display(Image.open(f"Images/{r['filename']}"))

In [ ]:
# Image similarity search with custom reference
# Update with specific image from your dataset
custom_reference = "Images/" + image_files[5]  # Change index to explore

print(f"Reference: {os.path.basename(custom_reference)}")
display(Image.open(custom_reference))

similar = find_similar_panels(custom_reference, limit=4)
print("\nSimilar panels:")
for s in similar[1:]:
    print(f"\n{s['filename']}")
    display(Image.open(f"Images/{s['filename']}"))

## Collection Statistics

View information about the current dataset

In [ ]:
collection = client.collections.get("ElectricalPanel")
agg = collection.aggregate.over_all()

print("ELECTRICAL PANEL COLLECTION STATISTICS")
print("="*60)
print(f"Total panels in database: {agg.total_count}")
print(f"Images directory: {image_dir}")
print(f"Vector database: Weaviate (meter-image-weaviate)")
print(f"Embedding model: CLIP (multi2vec-clip)")

## Next Steps

**POC Findings to Document:**
1. How well does CLIP identify similar panel configurations?
2. Can we distinguish panel types (meter vs breaker vs main)?
3. What patterns emerge in compatible vs incompatible groupings?
4. Do we need labeled training data for better classification?

**Enhancements to Consider:**
- Add manual labels for panel_type and compatibility_tag
- Integrate OCR to extract text from panels
- Calculate distance scores for compatibility thresholds
- Build classification model for panel types
- Add capacity calculation logic

In [ ]:
# Cleanup - close client connection
# client.close()
# print("Connection closed")

## Interactive Compatibility Analyzer

Gradio interface for uploading panel images and analyzing compatibility with existing panels in the database

In [ ]:
%pip install gradio

In [ ]:
import gradio as gr
from PIL import Image
import tempfile
import os

def analyze_panel_compatibility(uploaded_image):
    """
    Analyze an uploaded panel image for compatibility with database panels
    Returns similar (compatible) and dissimilar (incompatible) panels
    """
    if uploaded_image is None:
        return None, None, None, "Please upload an image"
    
    try:
        # Save uploaded image temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as tmp_file:
            uploaded_image.save(tmp_file.name, format='JPEG')
            temp_path = tmp_file.name
        
        # Encode image for Weaviate
        with open(temp_path, "rb") as file:
            query_image = base64.b64encode(file.read()).decode('utf-8')
        
        # Search for similar panels
        collection = client.collections.get("ElectricalPanel")
        res = collection.query.near_image(
            near_image=query_image,
            limit=10
        )
        
        # Clean up temp file
        os.unlink(temp_path)
        
        if len(res.objects) == 0:
            return None, None, None, "No panels found in database. Please load panels first."
        
        # Get top 3 compatible (most similar)
        compatible_panels = []
        for obj in res.objects[:3]:
            filename = obj.properties.get("filename")
            if filename:
                try:
                    img = Image.open(f"Images/{filename}")
                    compatible_panels.append(img)
                except Exception as e:
                    print(f"Error loading {filename}: {e}")
        
        # Get bottom 3 incompatible (least similar)
        incompatible_panels = []
        for obj in res.objects[-3:]:
            filename = obj.properties.get("filename")
            if filename:
                try:
                    img = Image.open(f"Images/{filename}")
                    incompatible_panels.append(img)
                except Exception as e:
                    print(f"Error loading {filename}: {e}")
        
        # Prepare result summary
        result_text = f"""
        ✓ Analysis Complete
        
        Found {len(res.objects)} panels in database for comparison
        
        🔍 COMPATIBLE PANELS (Top 3 Most Similar):
        {chr(10).join([f"  • {res.objects[i].properties.get('filename')}" for i in range(min(3, len(res.objects)))])}
        
        ⚠️  INCOMPATIBLE PANELS (Top 3 Least Similar):
        {chr(10).join([f"  • {res.objects[i].properties.get('filename')}" for i in range(max(0, len(res.objects)-3), len(res.objects))])}
        
        Recommendation: Panels showing high similarity may share compatible configurations,
        mounting patterns, or electrical specifications.
        """
        
        # Pad results to ensure we have 3 images each
        while len(compatible_panels) < 3:
            compatible_panels.append(None)
        while len(incompatible_panels) < 3:
            incompatible_panels.append(None)
        
        return (
            compatible_panels[0], compatible_panels[1], compatible_panels[2],
            incompatible_panels[0], incompatible_panels[1], incompatible_panels[2],
            result_text
        )
        
    except Exception as e:
        error_msg = f"Error analyzing image: {str(e)}\n\nMake sure Weaviate is running and panels are loaded."
        return None, None, None, None, None, None, error_msg

In [ ]:
# Create and launch Gradio interface
with gr.Blocks(title="Electrical Panel Compatibility Analyzer") as demo:
    gr.Markdown("# 🔌 Electrical Panel Compatibility Analyzer")
    gr.Markdown("""
    Upload an electrical panel image to analyze its compatibility with panels in the database.
    The system uses visual similarity to identify compatible and incompatible configurations.
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(
                label="Upload Panel Image", 
                type="pil",
                height=400
            )
            analyze_btn = gr.Button("🔍 Analyze Compatibility", variant="primary", size="lg")
    
    gr.Markdown("## Results")
    
    result_summary = gr.Textbox(
        label="Analysis Summary",
        lines=12,
        max_lines=15
    )
    
    gr.Markdown("### 🔍 Compatible Panels (High Similarity)")
    with gr.Row():
        compatible_1 = gr.Image(label="Most Compatible #1", type="pil")
        compatible_2 = gr.Image(label="Most Compatible #2", type="pil")
        compatible_3 = gr.Image(label="Most Compatible #3", type="pil")
    
    gr.Markdown("### ⚠️ Incompatible Panels (Low Similarity)")
    with gr.Row():
        incompatible_1 = gr.Image(label="Least Compatible #1", type="pil")
        incompatible_2 = gr.Image(label="Least Compatible #2", type="pil")
        incompatible_3 = gr.Image(label="Least Compatible #3", type="pil")
    
    # Connect the analyze button
    analyze_btn.click(
        fn=analyze_panel_compatibility,
        inputs=image_input,
        outputs=[
            compatible_1, compatible_2, compatible_3,
            incompatible_1, incompatible_2, incompatible_3,
            result_summary
        ]
    )
    
    gr.Markdown("""
    ---
    **How it works:**
    - Images are embedded using CLIP (multi2vec-clip) model
    - Vector similarity search identifies similar panel configurations
    - Compatible panels share visual patterns (mounting, layout, components)
    - Incompatible panels differ significantly in configuration
    
    **Note:** This is a proof-of-concept. For production use, add labeled training data
    and domain-specific classification models.
    """)

# Launch the interface
demo.launch(share=False, server_port=7860)